<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/02_modeling_exp/3rd_convnext_add_branch_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install timm # 현재 코랩에 이미 설치되어 있어서 주석처리 함

In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
from collections import defaultdict
import gc

In [3]:
# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
# src 폴더를 인식할 수 있도록 경로 추가 (현재 경로가 /content라면)
sys.path.append('/content/drive/MyDrive/HECTO')

from src.utils import set_seed, log_to_csv
from src.dataset import DeepfakeDataset, seed_worker
from src.trainer import train_one_epoch, validate_video_level, save_checkpoint

In [5]:
# 로컬에서 압축 해제 (네트워크 지연 없음)
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/Celeb_frames_pp_final.zip -d /content/Celeb_Extract
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/DF_Frames_Final.zip -d /content/DF_Extract
!unzip -q /content/drive/MyDrive/HECTO/Dataset/final_files/FF_Frames_Final.zip -d /content/FF++_Extract

In [6]:
# 2. 체크포인트 저장 경로 설정 (본인의 드라이브 경로에 맞게 수정 가능)
CHECKPOINT_DIR = '/content/drive/MyDrive/HECTO/checkpoints/05_convnext_add_branch'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# 3. 데이터셋 경로
CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/total_final.csv'

print(f"✅ 체크포인트 저장 경로: {CHECKPOINT_DIR}")

✅ 체크포인트 저장 경로: /content/drive/MyDrive/HECTO/checkpoints/05_convnext_add_branch


In [7]:
# 시드 설정
EXPERIMENT_SEED = 42
set_seed(EXPERIMENT_SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 시드 {EXPERIMENT_SEED} 고정 완료 및 {device} 준비")

✅ 시드 42 고정 완료 및 cuda 준비


In [8]:
g = torch.Generator()
g.manual_seed(EXPERIMENT_SEED)

# 기본 증강 설정
IMG_SIZE = 224

train_transform = transforms.Compose([
    # 1. 기하학적 변환 (256x256 원본 해상도에서 수행)
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),

    # 2. 화질 및 색상 변환
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
    ], p=0.3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

    # 3. 센터 크롭 (Resize 대신 사용)
    # 256x256 이미지의 중앙에서 224x224를 잘라내어 픽셀 왜곡을 방지합니다.
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),

    # 4. 텐서화 및 정규화
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

    # 5. 가려짐 대비 (ToTensor 이후 적용)
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

val_transform = transforms.Compose([
    # 1. 센터 크롭
    # 학습 시와 동일하게 중앙 영역을 224x224로 잘라냅니다.
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),

    # 2. 텐서화 및 정규화
    # 학습 시 사용한 파라미터와 완벽히 일치해야 합니다.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.val_acc_max = 0

    def __call__(self, val_acc):
        score = val_acc

        if self.best_score is None:
            self.best_score = score
            self.val_acc_max = val_acc
        # [수정] 정확도가 개선되지 않은 경우 (score가 best + delta보다 작을 때)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        # [수정] 정확도가 확실히 개선된 경우
        else:
            self.best_score = score
            self.val_acc_max = val_acc
            self.counter = 0

def load_checkpoint(model_name, model, optimizer, save_dir=CHECKPOINT_DIR):
    filename = os.path.join(save_dir, f"{model_name}_checkpoint.pth.tar")
    if os.path.isfile(filename):
        print(f"🔄 체크포인트 불러오는 중: {filename}")
        # map_location 추가하여 장치 불일치 방지
        checkpoint = torch.load(filename, map_location=device)

        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        es_state = checkpoint.get('es_state', None)

        # 만약 es_state에 best_score 정보가 없다면 best_acc로 강제 동기화 준비
        if es_state is not None:
            if 'best_score' not in es_state or es_state['best_score'] is None:
                es_state['best_score'] = best_acc

        return start_epoch, best_acc, es_state
    else:
        print("🆕 이전 체크포인트 없음. 처음부터 학습을 시작합니다.")
        return 0, 0, None

In [10]:
class MultiScaleConvNeXt(nn.Module):
    def __init__(self, model_name='convnext_tiny.fb_in1k', pretrained=True, checkpoint_path=None):
        super(MultiScaleConvNeXt, self).__init__()
        # 1. 백본 생성 (features_only=True로 설정하여 중간 특징 추출 가능하게 함)
        self.backbone = timm.create_model(model_name, pretrained=pretrained, features_only=True)

        # 2. 실험 1 가중치 로드 (백본 부분만 이식)
        if checkpoint_path and os.path.exists(checkpoint_path):
            print(f"📥 실험 1 가중치를 로드합니다: {checkpoint_path}")
            ckpt = torch.load(checkpoint_path, map_location='cpu')
            state_dict = ckpt.get('state_dict', ckpt)

            # backbone 부분만 필터링하여 로드
            new_state_dict = {}
            for k, v in state_dict.items():
                if 'backbone' in k:
                    # 'model.backbone.' 또는 'backbone.' 등의 접두어 제거
                    key = k.split('backbone.')[-1]
                    new_state_dict[key] = v

            # strict=False로 하여 새로 생긴 adapter/head 가중치는 제외하고 로드
            msg = self.backbone.load_state_dict(new_state_dict, strict=False)
            print(f"✅ 백본 가중치 로드 결과: {msg}")

        # 각 스테이지의 채널 수: Stage 2(192), Stage 3(384), Stage 4(768)
        # 3. 브랜치용 합성곱 레이어 (Adapter)
        self.adapter2 = nn.Sequential(
            nn.Conv2d(192, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU()
        )
        self.adapter3 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU()
        )
        self.adapter4 = nn.Sequential(
            nn.Conv2d(768, 768, kernel_size=3, padding=1),
            nn.BatchNorm2d(768),
            nn.ReLU()
        )

        self.gap = nn.AdaptiveAvgPool2d(1)

        # 4. 최종 분류기 (3개 스테이지의 특성을 합침: 192 + 384 + 768 = 1344)
        self.head = nn.Linear(192 + 384 + 768, 1)

    def forward(self, x):
        features = self.backbone(x) # [s1, s2, s3, s4]

        # 각 스테이지별 브랜치 통과 및 Global Average Pooling
        f2 = self.gap(self.adapter2(features[1])).flatten(1) # Stage 2
        f3 = self.gap(self.adapter3(features[2])).flatten(1) # Stage 3
        f4 = self.gap(self.adapter4(features[3])).flatten(1) # Stage 4

        # 특성 결합 및 최종 분류
        combined = torch.cat([f2, f3, f4], dim=1)
        return self.head(combined)

In [11]:
# 실험 대상 모델
model_names = ['convnext_tiny']

# 하이퍼파라미터
MAX_EPOCHS = 20  # 얼리스탑이 있으므로 넉넉히 잡습니다.
BATCH_SIZE = 32

# 데이터 로더 준비
train_dataset = DeepfakeDataset(CSV_PATH, split='train', transform=train_transform)
val_dataset = DeepfakeDataset(CSV_PATH, split='val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, worker_init_fn=seed_worker, generator=g)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [12]:
# 1. 결과 저장용 리스트
results_summary = []

for m_name in model_names:
    print(f"\n🚀 모델 학습 시작: {m_name} (Branch Structure + Differential LR)")

    # [설정] 실험 1에서 학습 완료된 베스트 가중치 경로 (본인의 경로에 맞게 수정)
    EXP1_BEST_PATH = "/content/drive/MyDrive/HECTO/checkpoints/03_convnext_different_lr/convnext_tiny_best.pth.tar"

    # 2. 모델 및 손실함수 초기화
    model = MultiScaleConvNeXt(model_name=m_name, checkpoint_path=EXP1_BEST_PATH).to(device)
    criterion = nn.BCEWithLogitsLoss()

    # 3. [핵심] AdamW 및 차등 학습률 적용
    # 백본(이미 학습됨)은 1e-6, 새로운 레이어들은 1e-4로 설정
    optimizer = optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': 1e-6},
        {'params': model.adapter2.parameters(), 'lr': 1e-4},
        {'params': model.adapter3.parameters(), 'lr': 1e-4},
        {'params': model.adapter4.parameters(), 'lr': 1e-4},
        {'params': model.head.parameters(), 'lr': 1e-4}
    ], weight_decay=0.05)

    # 4. 학습률 스케줄러 (전체적인 수렴 유도)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=MAX_EPOCHS)

    # 5. 얼리스탑 및 가중치 로드 로직 (기존 코드 유지)
    early_stopping = EarlyStopping(patience=5, verbose=True)
    # 인자 순서를 정의된 (model_name, model, optimizer, save_dir)에 맞춤
    # early_stopping은 함수 정의에 인자로 포함되어 있지 않으므로 내부 상태만 따로 업데이트해야 함
    start_epoch, best_acc, es_state = load_checkpoint(m_name, model, optimizer, CHECKPOINT_DIR)

    if es_state:
        early_stopping.__dict__.update(es_state)

    # --- 에포크 반복 시작 ---
    for epoch in range(start_epoch, MAX_EPOCHS):
        print(f"\n[Epoch {epoch+1}/{MAX_EPOCHS}]")

        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, video_acc = validate_video_level(model, val_loader, criterion, device)

        scheduler.step() # 스케줄러 업데이트

        is_best = video_acc > best_acc
        if is_best:
            best_acc = video_acc
            print(f"✨ Best Model Updated! (Acc: {best_acc:.4f})")

        early_stopping(video_acc)

        # 체크포인트 저장
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc': best_acc,
            'es_state': early_stopping.__dict__,
        }, is_best, m_name, CHECKPOINT_DIR)

        if early_stopping.early_stop:
            print(f"🛑 Early Stopping triggered at epoch {epoch+1}")
            break

    results_summary.append({'Model': m_name, 'Best Video Acc': best_acc})

    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

print("\n✅ 모든 모델 실험 완료!")


🚀 모델 학습 시작: convnext_tiny (Branch Structure + Differential LR)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

📥 실험 1 가중치를 로드합니다: /content/drive/MyDrive/HECTO/checkpoints/03_convnext_different_lr/convnext_tiny_best.pth.tar
✅ 백본 가중치 로드 결과: _IncompatibleKeys(missing_keys=['stem_0.weight', 'stem_0.bias', 'stem_1.weight', 'stem_1.bias', 'stages_0.blocks.0.gamma', 'stages_0.blocks.0.conv_dw.weight', 'stages_0.blocks.0.conv_dw.bias', 'stages_0.blocks.0.norm.weight', 'stages_0.blocks.0.norm.bias', 'stages_0.blocks.0.mlp.fc1.weight', 'stages_0.blocks.0.mlp.fc1.bias', 'stages_0.blocks.0.mlp.fc2.weight', 'stages_0.blocks.0.mlp.fc2.bias', 'stages_0.blocks.1.gamma', 'stages_0.blocks.1.conv_dw.weight', 'stages_0.blocks.1.conv_dw.bias', 'stages_0.blocks.1.norm.weight', 'stages_0.blocks.1.norm.bias', 'stages_0.blocks.1.mlp.fc1.weight', 'stages_0.blocks.1.mlp.fc1.bias', 'stages_0.blocks.1.mlp.fc2.weight', 'stages_0.blocks.1.mlp.fc2.bias', 'stages_0.blocks.2.gamma', 'stages_0.blocks.2.conv_dw.weight', 'stages_0.blocks.2.conv_dw.bias', 'stages_0.blocks.2.norm.weight', 'stages_0.blocks.2.norm.bias', 'stages_0.blo

EarlyStopping counter: 1 out of 5

[Epoch 16/20]


EarlyStopping counter: 2 out of 5

[Epoch 17/20]


EarlyStopping counter: 3 out of 5

[Epoch 18/20]


✨ Best Model Updated! (Acc: 0.9722)
⭐ [Best Updated] convnext_tiny - Best Acc: 0.9722 저장 완료

[Epoch 19/20]


EarlyStopping counter: 1 out of 5

[Epoch 20/20]


EarlyStopping counter: 2 out of 5

✅ 모든 모델 실험 완료!
